In [ ]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from io import StringIO
import requests
import os

ipaddress = os.environ['ipaddress']
dbname = os.environ['dbname']
username = os.environ['username']
password = os.environ['password']
port = os.environ['port']

def lambda_handler(event, context):
    
    tags = ['best_of_tasty','low_carb','pescatarian', 'contains_alcohol', 'indulgent_sweets',
        'healthy', 'dairy_free', 'comfort_food', 'kid_friendly', 'gluten_free', 'vegetarian', 
        'vegan']
        
    y = {}
    for t in tags:
        y[t] = get_data(t)
        y[t]["tag"] = t

    #Eine Liste aller Dataframes erstellen
    listOfDataframes = list(y.keys())
    
    # Der Loop geht über alle Keys des Dictionary und fügt die Values (Dataframes) zusammen
    listOfKeys = []
    for i in listOfDataframes:
        listOfKeys.append(y[i])
    df_total = pd.concat(listOfKeys)
    
    df = clean_data(df_total)
    
    # A long string that contains the necessary Postgres login information
    postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
    
    # Create the connection
    cnx = create_engine(postgres_str)
    
    # Load Dataframe into database
    df.head(0).to_sql('tasty', con=cnx, index=False, if_exists='append') # head(0) uses only the header
    # set index=False to avoid bringing the dataframe index in as a column 
    
    raw_con = cnx.raw_connection() # assuming you set up cnx as above
    cur  = raw_con.cursor()
    out = StringIO()
    
    # write just the body of your dataframe to a csv-like file object
    df.to_csv(out, sep='\t', header=False, index=False) 
    
    out.seek(0) # sets the pointer on the file object to the first line
    contents = out.getvalue()
    cur.copy_from(out, 'tasty', null="") # copies the contents of the file object into the SQL cursor and sets null values to empty strings
    raw_con.commit()
    
    #delete duplicates
    sql = """    
        DELETE FROM
        tasty x
        USING tasty y
        WHERE
        x.id = y.id
        AND x.ctid > y.ctid;
    """
    with cnx.begin() as connection:     
        connection.execute(sql)
    
    print("Data was successfully loaded into the DB.")
    
  
    


def get_data(tag):
    
    # credentials for Tasty
    headers = {
    'x-rapidapi-host': "tasty.p.rapidapi.com",
    'x-rapidapi-key': "151e12b103msha0b4622f262fc41p188e98jsn8ddd7233d384"
    }
    
    # query to select data from API
    url = "https://tasty.p.rapidapi.com/recipes/list"

    querystring = {"from":"0","size":"500","tags":tag}

    response = requests.request("GET", url, headers=headers, params=querystring)
    results = response.json()['results']
    results = pd.json_normalize(results)
    
    # create data frame
    df_recipes = pd.DataFrame(results)
    
    # create subset
    df = df_recipes[['name',
     'original_video_url',
     'topics',
     'keywords',
     'tags',
     'num_servings',
     'total_time_minutes',
     'yields',
     'country',
     'tips_and_ratings_enabled',
     'aspect_ratio',
     'credits',
     'sections',
     'instructions',
     'id',
     'prep_time_minutes',
     'description',
     'cook_time_minutes',
     'nutrition.fiber',
     'nutrition.protein',
     'nutrition.fat',
     'nutrition.calories',
     'nutrition.sugar',
     'nutrition.carbohydrates',
     'user_ratings.count_positive',
     'user_ratings.score',
     'user_ratings.count_negative',
     'total_time_tier.display_tier']]
     
    df = df.rename(columns={'nutrition.fiber': 'nutrition_fiber', 'nutrition.protein': 'nutrition_protein', 'nutrition.fat': 'nutrition_fat', \
        'nutrition.calories': 'nutrition_calories', 'nutrition.sugar': 'nutrition_sugar', 'nutrition.carbohydrates': 'nutrition_carbohydrates', \
        'user_ratings.count_positive': 'user_ratings_count_positive', 'user_ratings.score': 'user_ratings_score', 'user_ratings.count_negative': 'user_ratings_count_negative', \
        'total_time_tier.display_tier': 'total_time_tier_display_tier'})
    
    # fillna
    df.fillna(0)
    
    
    return df
    
def clean_data(df):
    
    df = df[df['instructions'].notna()]
    
    for index_label, row_series in df.iterrows():
        l = []
        for i in row_series['topics']:
            l.append(i['name'])
        df.at[index_label , 'topics_clean'] = ', '.join([str(item) for item in l])
        
    for index_label, row_series in df.iterrows():
        l = []
        for i in row_series['tags']:
            l.append(i['name'])
        df.at[index_label , 'tags_clean'] = ', '.join([str(item) for item in l])
        
    for index_label, row_series in df.iterrows():
        l = []
        for i in row_series['credits']:
            l.append(i['type'])
        # only the first value as we need only one value per entry    
        df.at[index_label , 'credits_clean'] = l[0]
        
    for index_label, row_series in df.iterrows():
        l = []
        for i in row_series['instructions']:
            l.append(i['display_text'])
        df.at[index_label , 'instructions_clean'] = ', '.join([str(item) for item in l])
        
    for index_label, row_series in df.iterrows():
        l = []
        for i in row_series['sections']:
            for j in i['components']:
                l.append(j['raw_text'])
        df.at[index_label , 'ingredients'] = ', '.join([str(item) for item in l])
        
    df.drop(['topics', 'tags', 'credits', 'instructions', 'sections'], axis=1, inplace=True)
    
    return df
    